In [ ]:
# Importar las librerías necesarias para el análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import warnings

# Ignorar advertencias (warnings) para mantener una salida limpia
warnings.filterwarnings("ignore")

# Cargar el dataframe limpio.
df_cleaned = pd.read_csv('datos/df_cleaned.csv')

# Haremos una copia para mantener el original intacto.
df_featured = df_cleaned.copy()

### Sección 5: Ingeniería de Características (con Discretización)

#### Resumen Ejecutivo de la Sección

Esta sección transforma el `df_cleaned` en un formato listo para el modelado, con un enfoque en la **discretización** de variables continuas. Las transformaciones clave son:

1.  **Discretización mediante Codificación Ordinal Dinámica:** Las variables continuas de medición se discretizaron utilizando las clasificaciones de calidad. Para evitar un mapa codificado a mano, se **reutilizó la lógica del EDA para generar los mapas ordinales dinámicamente**. Se parseó el archivo de metadatos (`Escalas_subterranea.csv`) para crear un mapa de clasificación para cada `CALIDAD_*` basado en el orden de las reglas, asegurando que el ranking numérico refleje con precisión las reglas de negocio.
2.  **One-Hot Encoding de `CONTAMINANTES`:** La columna de texto `CONTAMINANTES` se transformó en columnas binarias para cada tipo de contaminante.
3.  **Eliminación de Columnas Redundantes y No Informativas:** Se eliminaron las columnas de alta cardinalidad, identificadores, y las columnas de medición continua originales, ya que ahora están representadas por sus contrapartes ordinales.

El resultado es el DataFrame `df_featured`, un conjunto de datos numérico donde las mediciones continuas clave han sido reemplazadas por bins ordinales significativos y generados de forma programática.

In [ ]:
# --- 1. Discretización: Generación Dinámica de Mapas Ordinales ---

# Paso 1.1: Reutilizar la lógica del EDA para parsear los metadatos y obtener las reglas de negocio
criteria_file_path = 'datos/Escalas_subterranea.csv'
with open(criteria_file_path, 'r', encoding='latin-1') as f:
    criteria_raw_text = f.read()

header_word_map = {
    'COLIFORMES': 'COLI_FEC', 'CROMO': 'CR', 'CONDUCTIVIDAD': 'CONDUC',
    'CADMIO': 'CD', 'ARSENICO': 'AS', 'ALCALINIDAD': 'ALC', 'PLOMO': 'PB',
    'NITRATOS': 'N_NO3', 'MANGANESO': 'MN', 'MERCURIO': 'HG',
    'FLUORUROS': 'FLUO', 'HIERRO': 'FE', 'DUREZA': 'DUR'
}

def parse_criteria(criteria_string):
    lines = criteria_string.strip().split('\n')
    parsed_rules = {}
    current_param_key = None
    for line in lines:
        parts = [p.strip() for p in line.split(',', 2)]
        if parts[0].startswith('CALIDAD DEL AGUA PARA'):
            header_text = parts[0]
            current_param_key = None
            if "SALINIZACION" in header_text: current_param_key = "SDT_salin"
            elif "RIEGO AGRICOLA" in header_text: current_param_key = "SDT_ra"
            else:
                for word, key in header_word_map.items():
                    if word in header_text: current_param_key = key; break
            if current_param_key: parsed_rules[current_param_key] = []
        elif current_param_key and len(parts) > 1 and not parts[0].lower() == 'criterio':
            parsed_rules[current_param_key].append({'label': parts[0], 'text': parts[1]})
    return parsed_rules

quality_rules = parse_criteria(criteria_raw_text)

# Paso 1.2: Generar un diccionario de mapas ordinales a partir de las reglas parseadas
all_ordinal_maps = {}
for param, rules in quality_rules.items():
    # El mapa ordinal se crea usando el índice de la regla como el rango (0=mejor, 1=siguiente, etc.)
    ordinal_map = {rule['label']: i for i, rule in enumerate(rules)}
    all_ordinal_maps[f"CALIDAD_{param}"] = ordinal_map
    # Caso especial para SDT que tiene dos clasificaciones
    if param == "SDT_ra": all_ordinal_maps["CALIDAD_SDT_salin"] = all_ordinal_maps.pop("CALIDAD_SDT_ra")

print("Se generaron mapas ordinales dinámicamente desde los metadatos.")

# Paso 1.3: Aplicar los mapas ordinales correspondientes a cada columna
quality_cols = [col for col in df_featured.columns if col.startswith('CALIDAD_')]
for col in quality_cols:
    if col in all_ordinal_maps:
        df_featured[col] = df_featured[col].map(all_ordinal_maps[col])
    else: # Manejar columnas como CALIDAD_AS que no coinciden directamente
        param_key = col.split('_')[1]
        if f"CALIDAD_{param_key}" in all_ordinal_maps:
             df_featured[col] = df_featured[col].map(all_ordinal_maps[f"CALIDAD_{param_key}"])

# Paso 1.4: Verificación de NaN
nan_check = df_featured[quality_cols].isnull().sum().sum()
if nan_check > 0:
    print(f"ADVERTENCIA: Se encontraron {nan_check} valores NaN. Revise la lógica de mapeo.")
else:
    print("Verificación exitosa: No se generaron valores NaN durante la codificación ordinal.")

# --- 2. One-Hot Encoding y otras transformaciones ---
contaminant_list = df_featured['CONTAMINANTES'].str.split(', ').explode().unique()
unique_contaminants = sorted([c for c in contaminant_list if c != 'Sin Contaminantes'])
for contaminant in unique_contaminants:
    new_col_name = f"CONTAMINANTE_{contaminant}"
    df_featured[new_col_name] = df_featured['CONTAMINANTES'].apply(lambda x: 1 if contaminant in x else 0)

cols_to_drop = [
    'CLAVE', 'SITIO', 'ORGANISMO_DE_CUENCA', 'ESTADO', 'MUNICIPIO', 'ACUIFERO', 'SUBTIPO', 'PERIODO',
    'CONTAMINANTES',
    'ALC_mg/L', 'CONDUCT_mS/cm', 'SDT_M_mg/L', 'FLUORUROS_mg/L', 'DUR_mg/L', 'COLI_FEC_NMP/100_mL', 
    'N_NO3_mg/L', 'AS_TOT_mg/L', 'CD_TOT_mg/L', 'CR_TOT_mg/L', 'HG_TOT_mg/L', 'PB_TOT_mg/L', 'MN_TOT_mg/L', 'FE_TOT_mg/L'
]
df_featured.drop(columns=cols_to_drop, inplace=True, errors='ignore')

compliance_cols = [col for col in df_featured.columns if 'CUMPLE_CON' in col or 'SEMAFORO' in col]
binary_mapping = {'SI': 0, 'NO': 1, 'VERDE': 0, 'AMARILLO': 1, 'ROJO': 1}
for col in compliance_cols:
    df_featured[col] = df_featured[col].map(binary_mapping)

# --- Verificación Final ---
print(f"\nForma final del DataFrame: {df_featured.shape}")
print("Vista previa del DataFrame transformado:")
display(df_featured.head())